In [1]:
# 必要なライブラリをインポート
import pandas as pd
import os

# --- 1. データの読み込み ---
cleaned_data_path = '../data/processed/cleaned_data.parquet'
df = pd.read_parquet(cleaned_data_path)

In [2]:
#建築年を西暦に
# '戦前'を1945に変換
df['建築年'] = df['建築年'].replace('戦前', 1945)
def convert_wareki_to_seireki(wareki):
    """
    和暦を西暦に変換する関数
    """
    if pd.isna(wareki):
        return None

    wareki = str(wareki).strip()
    
    # 昭和
    if '昭和' in wareki:
        year_str = wareki.replace('昭和', '').replace('年', '')
        try:
            year = int(year_str)
            return 1925 + year
        except ValueError:
            return None
    
    # 平成
    elif '平成' in wareki:
        year_str = wareki.replace('平成', '').replace('年', '')
        try:
            year = int(year_str)
            return 1988 + year
        except ValueError:
            return None
    
    # 令和
    elif '令和' in wareki:
        year_str = wareki.replace('令和', '').replace('年', '')
        try:
            year = int(year_str)
            return 2018 + year
        except ValueError:
            return None
    
    # 大正
    elif '大正' in wareki:
        year_str = wareki.replace('大正', '').replace('年', '')
        try:
            year = int(year_str)
            return 1911 + year
        except ValueError:
            return None
            
    # 明治
    elif '明治' in wareki:
        year_str = wareki.replace('明治', '').replace('年', '')
        try:
            year = int(year_str)
            return 1867 + year
        except ValueError:
            return None

    # 和暦でない場合はそのまま返すか、NaNにする
    return None

df['建築年'] = df['建築年'].apply(convert_wareki_to_seireki)

#取引時点を西暦に
df['取引時点'] = df['取引時点'].str.extract(r'(\d{4})').astype(float)

#取引時点での築年数の追加
df['取引時点での築年数'] = df['取引時点'] - df['建築年']


#欠損値削除
df.dropna(subset=['取引時点での築年数'], inplace=True)

In [5]:
import re
# 新しく追加する人口密度のテキストデータ
density_text = """
1東京都6,451.17
2大阪府4,603.02
3神奈川県3,816.89
4埼玉県1,929.89
5愛知県1,443.06
6千葉県1,217.00
7福岡県1,022.06
8沖縄県642.85
9兵庫県635.26
10京都府546.65
11香川県488.61
12茨城県460.79
13静岡県453.15
14滋賀県348.69
15奈良県348.18
16佐賀県322.73
17広島県320.44
18宮城県308.58
19長崎県302.75
20群馬県296.97
21三重県296.37
22栃木県293.74
23石川県262.42
24岡山県257.31
25富山県234.48
26熊本県228.92
27愛媛県224.70
28山口県209.32
29和歌山県186.18
30岐阜県180.12
31山梨県176.97
32福井県176.27
33大分県171.15
34新潟県166.79
35鹿児島県166.58
36徳島県165.27
37鳥取県151.43
38長野県146.62
39宮崎県133.35
40福島県126.40
41青森県120.76
42山形県108.42
43島根県95.62
44高知県92.32
45秋田県77.02
46岩手県74.92
47北海道64.29
"""

# --- Step 2: 人口密度データを整形してデータフレームを作成 ---

prefectures_density = []
densities = []

# テキストを行ごとに分割して処理
for line in density_text.strip().split('\n'):
    # 正規表現で都道府県名と数値を抽出
    match = re.search(r'\d+([^\d,.]+)\s*([\d,.]+)', line)
    if match:
        prefecture = match.group(1).strip()
        density_str = match.group(2)

        # コンマを削除して浮動小数点数に変換
        density_float = float(density_str.replace(',', ''))

        prefectures_density.append(prefecture)
        densities.append(density_float)

# 人口密度のデータフレームを作成
density_df = pd.DataFrame({
    '都道府県名': prefectures_density,
    '人口密度': densities
})


# --- Step 3: 最終的な結合 ---

# 坪単価が入ったdfに、さらに人口密度のデータを結合
df = pd.merge(df, density_df, on='都道府県名', how='left')
df


,ID,種類,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点での築年数,人口密度
0,1060685,中古マンション等,1108,北海道,札幌市厚別区,大谷地東,大谷地,8.0,３ＬＤＫ,80,...,None,準工業地域,60.0,200.0,2009.0,未改装,None,7.079181,14.0,64.29
1,1005580,中古マンション等,1101,北海道,札幌市中央区,南９条西,中島公園,5.0,１ＤＫ,30,...,住宅,近隣商業地域,80.0,300.0,2018.0,未改装,None,6.755875,36.0,64.29
2,1001363,中古マンション等,1101,北海道,札幌市中央区,北３条西,西１１丁目,11.0,３ＬＤＫ,65,...,None,近隣商業地域,80.0,300.0,2010.0,未改装,None,7.204120,19.0,64.29
3,1052374,中古マンション等,1108,北海道,札幌市厚別区,厚別中央２条,ひばりが丘(北海道),5.0,４ＬＤＫ,90,...,住宅,商業地域,80.0,400.0,2015.0,未改装,None,7.591065,0.0,64.29
4,1059107,中古マンション等,1108,北海道,札幌市厚別区,厚別東４条,新さっぽろ,12.0,４ＬＤＫ,80,...,None,準工業地域,60.0,200.0,2012.0,未改装,None,6.875061,30.0,64.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551640,47001698,中古マンション等,47201,沖縄県,那覇市,松島,市立病院前(沖縄),3.0,４ＬＤＫ,80,...,住宅,第１種住居地域,60.0,200.0,2017.0,改装済,None,7.518514,10.0,642.85
551641,47030340,中古マンション等,47201,沖縄県,那覇市,前島,美栄橋,4.0,２ＤＫ,40,...,None,商業地域,80.0,400.0,2008.0,未改装,None,5.832509,24.0,642.85
551642,47000640,中古マンション等,47201,沖縄県,那覇市,字国場,壺川,29.0,３ＬＤＫ,65,...,住宅,第１種中高層住居専用地域,60.0,200.0,2018.0,改装済,None,7.301030,22.0,642.85
551643,47014718,中古マンション等,47201,沖縄県,那覇市,小禄,奥武山公園,15.0,３ＬＤＫ,65,...,住宅,第２種中高層住居専用地域,60.0,200.0,2014.0,未改装,None,7.322219,9.0,642.85


In [6]:

print("--- 特徴量作成後のデータ ---")
print(df.head())

# --- 3. 処理後のデータを保存 ---
processed_dir = '../data/processed/'
processed_file = 'features.parquet' # モデル学習用の最終データ
processed_path = os.path.join(processed_dir, processed_file)

df.to_parquet(processed_path, index=False)

print(f"\n特徴量作成後のデータを {processed_path} に保存しました。")

--- 特徴量作成後のデータ ---
        ID        種類  市区町村コード 都道府県名   市区町村名     地区名      最寄駅：名称  最寄駅：距離（分）  \
0  1060685  中古マンション等     1108   北海道  札幌市厚別区    大谷地東         大谷地        8.0   
1  1005580  中古マンション等     1101   北海道  札幌市中央区    南９条西        中島公園        5.0   
2  1001363  中古マンション等     1101   北海道  札幌市中央区    北３条西       西１１丁目       11.0   
3  1052374  中古マンション等     1108   北海道  札幌市厚別区  厚別中央２条  ひばりが丘(北海道)        5.0   
4  1059107  中古マンション等     1108   北海道  札幌市厚別区   厚別東４条       新さっぽろ       12.0   

    間取り  面積（㎡）  ...  今後の利用目的    都市計画 建ぺい率（％） 容積率（％）    取引時点   改装  取引の事情等  \
0  ３ＬＤＫ     80  ...     None   準工業地域    60.0  200.0  2009.0  未改装    None   
1   １ＤＫ     30  ...       住宅  近隣商業地域    80.0  300.0  2018.0  未改装    None   
2  ３ＬＤＫ     65  ...     None  近隣商業地域    80.0  300.0  2010.0  未改装    None   
3  ４ＬＤＫ     90  ...       住宅    商業地域    80.0  400.0  2015.0  未改装    None   
4  ４ＬＤＫ     80  ...     None   準工業地域    60.0  200.0  2012.0  未改装    None   

   取引価格（総額）_log 取引時点での築年数   人口密度  
0      7.079181     